In [2]:
import pandas as pd,numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,cohen_kappa_score,confusion_matrix,recall_score,precision_score
from sklearn.linear_model import LogisticRegression
import plotly.express as px
from datetime import datetime
import time, random
import xgboost as xgb
#tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
train = pd.read_csv("https://raw.githubusercontent.com/Layden23/bet369/master/Datasets/train2.csv", header = 0)

In [4]:
len(train["MatchID"].unique())

1679

In [5]:
train.isna().sum()

MatchID                    18
LeagueID                   18
LeagueName                 18
LeagueFullname             18
LeagueAbbreviation         18
CountryID                  18
CountryName                18
CountryAbbreviation        18
HomeTeamID                 18
HomeTeamName               18
GuestTeamID                18
GuestTeamName              18
HostHandicapFT             75
ExpectedGoalsFT            77
BettingCuoteFT         151720
HostHandicapHT            274
ExpectedGoalsHT           268
BettingCuoteHT         151870
HostGoalsFT                18
GuestGoalsFT               18
HostCornersFT              18
GuestCornersFT             18
HostYCFT                   18
GuestYCFT                  18
HostRCFT                   18
GuestRCFT                  18
HostAttacksFT              18
GuestAttacksFT             18
HostDAttacksFT             18
GuestDAttacksFT            18
HostonTargetFT             18
GuestonTargetFT            18
HostoffTargetFT            18
GuestoffTa

In [6]:
train = train.drop(train[train["MatchID"] == "MatchID"].index)

In [8]:
#Remove Strange Matches
train = train.loc[(train['HostPossessionFT'] != "0") & (train['GuestPossessionFT'] != "0")]
#Fix Goal Problem
train["Goal"] = pd.to_numeric(train["HostGoalsFT"]) + pd.to_numeric(train["GuestGoalsFT"])

In [14]:
train = train.reset_index(drop=True)
del train["level_0"]
del train["index"]
train

,MatchID,LeagueID,LeagueName,LeagueFullname,LeagueAbbreviation,CountryID,CountryName,CountryAbbreviation,HomeTeamID,HomeTeamName,...,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal
0,684112,3,AUS AL,Australia A-League,A,7,Australia,A,350,Brisbane Roar,...,41,36,2,2,4,4,64,36,50,1.0
1,685779,842,ISR LBS,Israel Liga Bet South,I,97,Israel,Y,16763,Hapoel Ironi Gedera,...,46,40,1,2,6,0,40,60,44,0.0
2,686017,2769,RUS U20 L,Russia U20 League,R,12,Russia,E,36457,Rubin Kazan U20,...,25,39,3,3,7,4,50,50,43,2.0
3,685849,220,CSK YL,Czech Youth League,C,10,the Czech Republic,J,1225,FK Pribram U19,...,22,23,5,3,1,0,47,53,35,4.0
4,684112,3,AUS AL,Australia A-League,A,7,Australia,A,350,Brisbane Roar,...,42,36,2,2,4,4,65,35,51,1.0
5,685779,842,ISR LBS,Israel Liga Bet South,I,97,Israel,Y,16763,Hapoel Ironi Gedera,...,46,42,1,3,6,0,40,60,44,0.0
6,686017,2769,RUS U20 L,Russia U20 League,R,12,Russia,E,36457,Rubin Kazan U20,...,25,40,3,3,7,5,50,50,44,2.0
7,685849,220,CSK YL,Czech Youth League,C,10,the Czech Republic,J,1225,FK Pribram U19,...,23,23,5,3,1,0,47,53,36,4.0
8,684112,3,AUS AL,Australia A-League,A,7,Australia,A,350,Brisbane Roar,...,44,37,2,2,5,4,65,35,53,1.0
9,685779,842,ISR LBS,Israel Liga Bet South,I,97,Israel,Y,16763,Hapoel Ironi Gedera,...,48,43,1,3,6,0,40,60,HT,0.0


In [22]:
a = train[(train["Goal"] == "0") & (train["Status"] == "HT")].MatchID.unique()

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [24]:
train0 = train[train.MatchID.isin(a)]
train0 = train0.drop(train0[train0["Status"] == "NS"].index)
train0 = train0.drop(train0[train0["Status"] == "HT"].index)
train0 = train0.drop(train0[train0["Status"] == "FT"].index)
train0["Status"] = pd.to_numeric(train0["Status"])
train0 = train0[train0["Status"] <= 45]

In [32]:
b = train0[(train0["Goal"] == 0) & (train0["Status"] == 45)].MatchID.unique()

In [33]:
train00 = train0[train0.MatchID.isin(b)]

In [34]:
train00.sort_values("Goal")

,MatchID,LeagueID,LeagueName,LeagueFullname,LeagueAbbreviation,CountryID,CountryName,CountryAbbreviation,HomeTeamID,HomeTeamName,...,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal


In [ ]:
train0[train0["Goal"] == 1]
#There are matches with same IDs

In [ ]:
train[train["MatchID"] == "684371"].MatchID.unique()

In [ ]:
train[train["Status"] == "HT"].iloc[:,-20:].sort_values(by = "Goal", ascending = False)

In [ ]:
s = 69

In [ ]:
#Remove columns with lots of missing values
del train["BettingCuoteHT"]
del train["BettingCuoteFT"]
del train["HostGoalsFT"]
del train["GuestGoalsFT"]

In [ ]:
#Creating train with matches that score in the first half
# train.Status = train.Status.replace(["HT","FT"],[45,90])
train = train.drop(train[train["Status"] == "NS"].index)
train = train.drop(train[train["Status"] == "HT"].index)
train = train.drop(train[train["Status"] == "FT"].index)

#Transform variables to numeric
for i in train.iloc[:,12:].columns:
    train[i] = pd.to_numeric(train[i])
#Creating train with matches that don't score in the first half
train = train[(train["Status"] <= 45) & (train["Goal"] <= 1)]

In [ ]:
len(train.MatchID.unique())

In [ ]:
train0.Goal.unique()

In [ ]:
train1 = train[train["Goal"] == 1].sort_values(by = ["Status","MatchID"]).groupby("MatchID", as_index=False).first()
train1

In [ ]:
train = train1.append(train0)
train = train.dropna()
train = train.drop_duplicates()

In [ ]:
# Indicies of each class' observations
random.seed(s)
i_class0 = np.where(train["Goal"] == 0)[0]
i_class1 = np.where(train["Goal"] == 1)[0]

#Get size of the underrepresented class
n_class0 = len(i_class0)
n_class1 = len(i_class1)

i_class0_downsampled = np.random.choice(i_class0, size=n_class1, replace=False)
# i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)
train = train.iloc[i_class1,:].append(train.iloc[i_class0_downsampled,:])

In [ ]:
train.isna().sum()

In [ ]:
train.sort_values(by = "Goal", ascending = False)

In [ ]:
px.histogram(train[train["Goal"]== 0], x = "Status", nbins = 45)

In [ ]:
px.histogram(train[train["Goal"]== 1], x = "Status", nbins = 45)

In [ ]:
px.histogram(train, x = "Goal")

In [ ]:
train

# Feature Engineering

In [ ]:
X = train.iloc[:,12:33]
X.columns

In [ ]:
stand = MinMaxScaler().fit(X)
X = pd.DataFrame(stand.transform(X))
pca = PCA(0.99)
pca.fit(X)
pcX = pca.transform(X)
y = train["Goal"]
#Transform variables to numeric
for i in X.columns:
    X[i] = pd.to_numeric(X[i])
y = pd.to_numeric(y)


In [ ]:
random.seed(s)
X_train, X_test, y_train, y_test = train_test_split(pcX,y,train_size = 0.7)

In [ ]:
#Random Forest
random.seed(s)
rf = RandomForestClassifier(n_estimators = 200)
rf.fit(X_train, y_train)


In [ ]:
# Logistic Regression
random.seed(s)
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
# XGBTree 
# random.seed(s)
# cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
# ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
#              'objective': 'binary:logistic'}
# xgb = GridSearchCV(xgb.XGBClassifier(**ind_params), 
#                             cv_params, 
#                              scoring = 'accuracy', cv = 5, n_jobs = -1)

In [ ]:
xgb = xgb.XGBClassifier()
xgb.fit(X_train,y_train)

In [ ]:
#Neuronal network
tf.keras.backend.clear_session() # For easy reset of notebook state.
#### Ignacio: You MUST explain a bit your neural network topology: 
#### One hidden layer with 128 neurons with the "Relux" activation function 
#### which is better for computational efficiency. Then you add an output layer with 3 neurons (one for each building you want to predict).
#### As the output neurons give you some value between 0-1, you apply the softmax function in order to normalize those outputs and interpret them 
#### as probabilities of being in each building.
nmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.softmax)
])

#### Ignacio: Why this choices of optimizer and loss function and metrics?
nmodel.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
nmodel.fit(np.array(X_train), np.array(y_train), epochs=10)

In [ ]:
test_loss, test_acc = nmodel.evaluate(X_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
#Predictions
rfpreds = rf.predict(X_test)
lrpreds = lr.predict(X_test)
xgbpreds = xgb.predict(X_test)
npreds = np.round(nmodel.predict(X_test)[:,1])

In [ ]:
#Probabilities
rfprobs = rf.predict_proba(X_test)[:,1]
lrprobs = lr.predict_proba(X_test)[:,1]
xgbprobs = xgb.predict_proba(X_test)[:,1]
nprobs = nmodel.predict(X_test)[:,1]

In [ ]:
# importances = rf.feature_importances_
# cols = X.columns
# pd.DataFrame({"Variable": cols,"Importance" : importances})

In [ ]:
results = pd.DataFrame({"Goal": y_test,"rfpreds" : rfpreds,"lrpreds": lrpreds, "xgbpreds": xgbpreds,
              "rfprobs": rfprobs, "lrprobs": lrprobs, "xgbprobs": xgbprobs, "npreds": npreds, "nprobs": nprobs})

In [ ]:
results

In [ ]:
rf_test = results[results["rfprobs"] >= 0.9]
print(accuracy_score(rf_test["Goal"],rf_test["rfpreds"]))
print(accuracy_score(rfpreds,y_test))

In [ ]:
rf_test

In [ ]:
lr_test = results[results["lrprobs"] >= 0.9]
print(accuracy_score(lr_test["Goal"],lr_test["lrpreds"]))
print(accuracy_score(lrpreds,y_test))

In [ ]:
lr_test

In [ ]:
xgb_test = results[results["xgbprobs"] >= 0.9]
print(accuracy_score(xgb_test["Goal"],xgb_test["xgbpreds"]))
print(accuracy_score(xgbpreds,y_test))

In [ ]:
xgb_test

In [ ]:
n_test = results[results["nprobs"] >= 0.9]
print(accuracy_score(n_test["Goal"],n_test["npreds"]))
print(accuracy_score(npreds,y_test))

In [ ]:
n_test

In [ ]:
confusion_matrix(xgbpreds,y_test)

In [ ]:
confusion_matrix(lrpreds,y_test)


In [ ]:
confusion_matrix(rfpreds,y_test)

In [ ]:
confusion_matrix(npreds,y_test)

In [ ]:
print("Metric   "," RF "," LR ","XGB","NN")
print("Accuracy ", round(accuracy_score(rfpreds,y_test),2), round(accuracy_score(lrpreds,y_test),2), round(accuracy_score(xgbpreds,y_test),2),round(accuracy_score(npreds, y_test),2))
print("Kappa    ",round(cohen_kappa_score(rfpreds,y_test),2),round(cohen_kappa_score(lrpreds,y_test),2),round(cohen_kappa_score(xgbpreds,y_test),2),round(cohen_kappa_score(npreds, y_test),2))
print("Precision",round(precision_score(rfpreds,y_test),2),round(precision_score(lrpreds,y_test),2),round(precision_score(xgbpreds,y_test),2),round(precision_score(npreds, y_test),2))
print("Recall   ", round(recall_score(rfpreds,y_test),2),round(recall_score(lrpreds,y_test),2),round(recall_score(xgbpreds,y_test),2),round(recall_score(npreds, y_test),2))

# Predictions Live

In [ ]:
filename = datetime.now().strftime('%Y-%m-%d/gambling-%Y-%m-%d-%H-%M.csv')
test1 = pd.read_csv(filename, header = 0)
del test1["BettingCuoteHT"]
del test1["BettingCuoteFT"]
del test1["HostGoalsFT"]
del test1["GuestGoalsFT"]
test = test1.dropna().copy()
test = test.loc[(test['HostPossessionFT'] != 0) & (test['GuestPossessionFT'] != 0)]
test = test.drop(test[test["Status"] == "HT"].index)
test = test.drop(test[test["Status"] == "NS"].index)
test = test.drop(test[test["Status"] == "FT"].index)
ntest = test.iloc[:,12:33].copy()
pctest = pca.transform(ntest)
tpreds = rf.predict(pctest)
tprobs = rf.predict_proba(pctest)[:,1]
test["Prediction"] = (np.round(xgb.predict(pctest)))
test["Probability"] = rf.predict_proba(pctest)[:,1]
test["Status"] = pd.to_numeric(test["Status"])
test0 = test[(test["Status"] <= 45) & (test["Goal"] == 0)]
test0 = test0[["HomeTeamName","GuestTeamName","Status","Goal","Prediction","Probability","HostonTargetFT","GuestonTargetFT","HostCornersFT","GuestCornersFT",
       "HostYCFT","GuestYCFT","HostRCFT",
      "GuestRCFT","HostAttacksFT","GuestAttacksFT","HostoffTargetFT",
      "GuestoffTargetFT","HostPossessionFT","GuestPossessionFT"]]

test0.sort_values("Probability", ascending = False)


In [ ]:
#Dependent of time
#Class balanced
#Minutes with Goal == 1, and random sample of minutes with Goal ==0
# 